# Diameter Monotonicity Analysis

This notebook analyzes CSV data from `results_perm1/` to find (coset, k, d) combinations where **diameter is NOT monotonically increasing** as n increases.

## What we're checking
For each unique (coset, k, d) combination, as we increase n, does the diameter always increase (or stay the same)? If diameter ever *decreases* when n increases, we flag it as a violation.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load all CSVs from results_perm1
results_dir = Path("results_perm1")
dfs = []

for group_dir in results_dir.iterdir():
    if group_dir.is_dir():
        csv_path = group_dir / "data.csv"
        if csv_path.exists():
            df = pd.read_csv(csv_path)
            df['group'] = group_dir.name
            dfs.append(df)
            print(f"Loaded {len(df)} rows from {group_dir.name}")

df = pd.concat(dfs, ignore_index=True)
print(f"\nTotal: {len(df)} rows from {len(dfs)} groups")
print(f"Columns: {list(df.columns)}")

Loaded 2298 rows from different
Loaded 281 rows from full_graph
Loaded 2138 rows from then
Loaded 2144 rows from repeats

Total: 6861 rows from 4 groups
Columns: ['coset', 'd', 'k', 'n', 'diameter', 'last_layer_size', 'total_states', 'growth', 'group', 'central']


In [2]:
def check_monotonicity(group_df):
    """
    Check if diameter is monotonically increasing as n increases.
    
    Returns dict with:
    - is_monotonic: True if diameter never decreases as n increases
    - violations: list of (n_prev, n_curr, diameter_prev, diameter_curr) where decrease happened
    - n_range: (min_n, max_n)
    - diameter_range: (min_diameter, max_diameter)
    """
    sorted_df = group_df.sort_values('n')
    diameters = sorted_df['diameter'].values
    n_values = sorted_df['n'].values
    
    violations = []
    for i in range(1, len(diameters)):
        if diameters[i] < diameters[i-1]:
            violations.append({
                'n_prev': int(n_values[i-1]),
                'n_curr': int(n_values[i]),
                'diameter_prev': int(diameters[i-1]),
                'diameter_curr': int(diameters[i]),
                'decrease': int(diameters[i-1] - diameters[i])
            })
    
    return {
        'is_monotonic': len(violations) == 0,
        'violations': violations,
        'n_range': (int(n_values.min()), int(n_values.max())),
        'diameter_range': (int(diameters.min()), int(diameters.max())),
        'diameter_series': list(zip(n_values.astype(int), diameters.astype(int)))
    }

print("Monotonicity check function defined.")

Monotonicity check function defined.


In [3]:
# Analyze all (coset, k, d) combinations
results = []

for (coset, k, d), group_df in df.groupby(['coset', 'k', 'd']):
    analysis = check_monotonicity(group_df)
    results.append({
        'coset': coset,
        'k': k,
        'd': d,
        'group': group_df['group'].iloc[0],
        'is_monotonic': analysis['is_monotonic'],
        'num_violations': len(analysis['violations']),
        'violations': analysis['violations'],
        'n_range': analysis['n_range'],
        'diameter_range': analysis['diameter_range'],
        'num_points': len(group_df),
        'diameter_series': analysis['diameter_series']
    })

results_df = pd.DataFrame(results)
print(f"Analyzed {len(results_df)} unique (coset, k, d) combinations")

Analyzed 1080 unique (coset, k, d) combinations


In [4]:
# Summary statistics
print("=" * 60)
print("SUMMARY")
print("=" * 60)
print(f"Total (coset, k, d) combinations: {len(results_df)}")
print(f"Monotonically increasing:         {results_df['is_monotonic'].sum()}")
print(f"NOT monotonically increasing:     {(~results_df['is_monotonic']).sum()}")
print()

# Breakdown by group
print("Breakdown by coset group:")
for group in results_df['group'].unique():
    group_results = results_df[results_df['group'] == group]
    non_mono = (~group_results['is_monotonic']).sum()
    total = len(group_results)
    print(f"  {group}: {non_mono}/{total} non-monotonic")

SUMMARY
Total (coset, k, d) combinations: 1080
Monotonically increasing:         398
NOT monotonically increasing:     682

Breakdown by coset group:
  repeats: 267/380 non-monotonic
  then: 263/380 non-monotonic
  different: 113/255 non-monotonic
  full_graph: 39/65 non-monotonic


In [5]:
# Display non-monotonic cases
non_monotonic = results_df[~results_df['is_monotonic']].copy()
non_monotonic = non_monotonic.sort_values('num_violations', ascending=False)

print(f"\n{'='*60}")
print(f"NON-MONOTONIC CASES ({len(non_monotonic)} total)")
print(f"{'='*60}\n")

for idx, row in non_monotonic.iterrows():
    print(f"coset={row['coset']}, k={row['k']}, d={row['d']} (group: {row['group']})")
    print(f"  n range: {row['n_range'][0]} to {row['n_range'][1]}")
    print(f"  diameter range: {row['diameter_range'][0]} to {row['diameter_range'][1]}")
    print(f"  {row['num_violations']} violation(s):")
    for v in row['violations']:
        print(f"    n={v['n_prev']} -> {v['n_curr']}: diameter {v['diameter_prev']} -> {v['diameter_curr']} (decrease of {v['decrease']})")
    print()


NON-MONOTONIC CASES (682 total)

coset=4Different, k=0, d=4 (group: different)
  n range: 5 to 20
  diameter range: 5 to 57
  8 violation(s):
    n=5 -> 6: diameter 6 -> 5 (decrease of 1)
    n=7 -> 8: diameter 12 -> 8 (decrease of 4)
    n=9 -> 10: diameter 19 -> 12 (decrease of 7)
    n=11 -> 12: diameter 27 -> 16 (decrease of 11)
    n=13 -> 14: diameter 35 -> 20 (decrease of 15)
    n=15 -> 16: diameter 42 -> 24 (decrease of 18)
    n=17 -> 18: diameter 50 -> 29 (decrease of 21)
    n=19 -> 20: diameter 57 -> 33 (decrease of 24)

coset=3Different, k=0, d=4 (group: different)
  n range: 5 to 20
  diameter range: 4 to 46
  8 violation(s):
    n=5 -> 6: diameter 5 -> 4 (decrease of 1)
    n=7 -> 8: diameter 9 -> 7 (decrease of 2)
    n=9 -> 10: diameter 14 -> 10 (decrease of 4)
    n=11 -> 12: diameter 19 -> 12 (decrease of 7)
    n=13 -> 14: diameter 25 -> 17 (decrease of 8)
    n=15 -> 16: diameter 32 -> 19 (decrease of 13)
    n=17 -> 18: diameter 39 -> 24 (decrease of 15)
    n=1

In [6]:
# Create a clean table of non-monotonic cases
if len(non_monotonic) > 0:
    table_df = non_monotonic[['group', 'coset', 'k', 'd', 'num_violations', 'n_range', 'diameter_range']].copy()
    table_df['n_range'] = table_df['n_range'].apply(lambda x: f"{x[0]}-{x[1]}")
    table_df['diameter_range'] = table_df['diameter_range'].apply(lambda x: f"{x[0]}-{x[1]}")
    display(table_df.reset_index(drop=True))
else:
    print("All (coset, k, d) combinations are monotonically increasing!")

,group,coset,k,d,num_violations,n_range,diameter_range
0,different,4Different,0,4,8,5-20,5-57
1,different,3Different,0,4,8,5-20,4-46
2,different,3Different,1,1,8,4-20,4-53
3,different,3Different,0,3,8,4-20,3-50
4,different,3Different,3,1,8,5-20,6-45
...,...,...,...,...,...,...,...
677,then,Binary0then1,3,9,1,13-15,27-55
678,then,Binary0then1,3,10,1,14-15,15-30
679,repeats,011Repeats,0,9,1,10-15,3-49
680,then,Binary0then1,4,5,1,10-15,11-70


In [7]:
# Plot diameter vs n for non-monotonic cases
if len(non_monotonic) > 0:
    # Limit to first 10 cases for readability
    cases_to_plot = non_monotonic.head(10)
    
    fig = go.Figure()
    
    for idx, row in cases_to_plot.iterrows():
        n_vals, diameter_vals = zip(*row['diameter_series'])
        label = f"{row['coset']}, k={row['k']}, d={row['d']}"
        
        fig.add_trace(go.Scatter(
            x=n_vals,
            y=diameter_vals,
            mode='lines+markers',
            name=label,
            hovertemplate=f"{label}<br>n=%{{x}}<br>diameter=%{{y}}<extra></extra>"
        ))
    
    fig.update_layout(
        title='Non-Monotonic Cases: Diameter vs n',
        xaxis_title='n',
        yaxis_title='Diameter',
        height=600,
        legend=dict(x=1.02, y=1)
    )
    fig.show()
else:
    print("No non-monotonic cases to plot.")

In [ ]:
# Export results to CSV
export_df = results_df[['group', 'coset', 'k', 'd', 'is_monotonic', 'num_violations', 'n_range', 'diameter_range', 'num_points']].copy()
export_df['n_range'] = export_df['n_range'].apply(lambda x: f"{x[0]}-{x[1]}")
export_df['diameter_range'] = export_df['diameter_range'].apply(lambda x: f"{x[0]}-{x[1]}")

export_path = "monotonicity_analysis_results.csv"
export_df.to_csv(export_path, index=False)
print(f"Results exported to {export_path}")
print(f"\nNon-monotonic cases can be filtered with: is_monotonic == False")